Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin

df = pd.read_csv('/datasets/insurance.csv')

In [2]:
display(df.head(), df.shape, df.isnull().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


(5000, 5)

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [3]:
X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']

display(X.shape, y.shape, X.head(), y.head())

(5000, 4)

(5000,)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказания $a$ не изменятся

**Обоснование:**

Представим выражение как $w_{new} = ((XM)^T XM)^{-1} (XM)^T y$, где $XM$ - это произведение признаков $X$ на обратимую матрицу $M$.


$$
w_{new} = ((XM)^T XM)^{-1} (XM)^T y = (X^T X M^T M)^{-1} X^T M^T y = (X^T X)^{-1} X^T (M^T M)^{-1} M^T y
$$


В левой части мы получили похожее на исходное выражение: $(X^T X)^{-1} X^T$

Поскольку матрица $M$ является обратимой, то $(M^T M)^{-1} M^T w = Mw = a_{new}$, где
$$
a_{new} = a
$$

## 3. Алгоритм преобразования

Напишем класс, в котором будет создаваться новая матрица.

Далее обучим модель из библиотеки sklean и проверим результат предсказания метрикой r2.

После чего проверим предсказания на соответствие с помощью numpy.isclose

## 4. Проверка алгоритма

In [4]:
class MatrixPrepare(TransformerMixin):
    
    def fit(self, X, y=None, random_state=None):
        if random_state:
            np.random.seed(random_state)
        
        X_sample = X.shape[1]
        M = np.random.normal(size=[X_sample, X_sample])
        try:
            M_inv = np.linalg.inv(M)
        except: 
            print('Matrix is not invertible')
        self.X = X
        self.M_inv = M_inv
        return self
    
    def transform(self, X, y=None):
        return X.dot(self.M_inv)

In [5]:
X_new = MatrixPrepare().fit_transform(X, random_state=42)
display(X_new.shape, X.shape, X_new.head())

(5000, 4)

(5000, 4)

,0,1,2,3
0,-25458.115200,16639.589185,-41399.411722,-11957.820064
1,-19501.837964,12741.928552,-31713.114724,-9167.092659
2,-10777.420742,7039.798675,-17525.724384,-5067.685356
3,-21403.595758,13996.811706,-34804.488829,-10046.799323
4,-13396.160295,8752.273743,-21785.410723,-6295.425578


In [6]:
linreg = LinearRegression()

linreg.fit(X, y)
predict1 = linreg.predict(X)

linreg.fit(X_new, y)
predict2 = linreg.predict(X_new)

print(np.isclose(r2_score(y, predict1), r2_score(y, predict2)))
print(r2_score(y, predict1), r2_score(y, predict2))

True
0.42494550286668 0.42494550286667376


Метрики одинаковы